In [290]:
import simpy
import random
import math
import numpy

In [291]:
RANDOM_SEED = 978
INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60


CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [292]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Time spent by a customer while it waits for the operator1 (Operator1 Queue waiting time)
queue_w_times_operator1 = []

# Time spent by a customer while it waits for the operator2 (operator2 Queue waiting time)
queue_w_times_operator2 = []

# The number of customer in the answering system
queue_w_customer = []


# Simulation end time
end_time = 0;



In [293]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        global end_time 
        prob=random.uniform(0,10)
        print('prob',prob)
        error=random.uniform(0,10)
        print('error',error)
        print('inital len',len(queue_w_customer)) 
        
       
        
        if len(queue_w_customer)<101:
            if error>1 :
                yield env.timeout(random.expovariate(CALL_RATE))
                if prob<3:    
                    yield self.env.process(self.call1())
                if prob>=3:    
                    yield self.env.process(self.call2())
            else:
                 print('%s A caller hangs up at %g' % (self.name, self.env.now))
        else:
            print('%s A caller drops without answering.-100 parallel channels- at %g' % (self.name, self.env.now))
            queue_w_customer.pop()
            print('drop len',len(queue_w_customer)) 


            
    def call1(self):
        
        with operator1.request() as req:
            queue_w_customer.pop()
            print('op1 len',len(queue_w_customer))
            yield req
            print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
            queue_w_times_operator1.append(self.env.now - self.arrival_t)
            yield self.env.process(self.get_help1_operator1())
            print('%s is done operator1 at %g' % (self.name, self.env.now))
            self.end_time = self.env.now
           
            

    def call2(self):
        
        with operator2.request() as req:
            yield req
            queue_w_customer.pop()
            print('op2 len',len(queue_w_customer))
            print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
            queue_w_times_operator2.append(self.env.now - self.arrival_t)
            yield self.env.process(self.get_help2_operator2())
            print('%s is done operator2 at %g' % (self.name, self.env.now))
            self.end_time = self.env.now
            

            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [294]:
class CustomerNorm(Customer):
    """ Customer that has normally distributed expert service"""
    def get_help1_operator1(self):
        duration = random.gauss(mu, sigma)
        while duration < 0:
            duration = random.gauss(mu, sigma)
            
        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [295]:

def customer_generator(env, operator1, operator2):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [296]:
     
def customer_norm_generator(env, operator1, operator2):
    """Generate new customer with normally distributed expert service that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = CustomerNorm('Customer %s' %(i+1), env, operator1, operator2)  
        

In [297]:
def break_generator(env, operator1, operator2):
    """Generate new break for expert."""
    BREAK_TIME_COUNTER1 = 0
    BREAK_TIME_COUNTER2 = 0
    while True:
        if len(queue_w_times_operator1) == CUSTOMER_COUNT:
            break
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        with operator1.request() as req:
            yield req
            print('Operator1 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator1 break done at %g' % (env.now))
            BREAK_TIME_COUNTER1+=1
            print('break1',BREAK_TIME_COUNTER1)
            
        if len(queue_w_times_operator2) == CUSTOMER_COUNT:
            break
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        with operator2.request() as req:
            yield req
            print('Operator2 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator2 break done at %g' % (env.now))
            BREAK_TIME_COUNTER2+=1
            print('break2',BREAK_TIME_COUNTER2)

In [298]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator1, operator2))
env.process(break_generator(env, operator1, operator2))
env.run()

Operator1 gives break at 10.1044
Operator1 break done at 13.1044
break1 1
Operator2 gives break at 18.9602
Operator2 break done at 21.9602
break2 1
Customer 1 initiated a call at 26.5136
prob 4.356679008537778
error 8.079502895221648
inital len 1
Customer 2 initiated a call at 31.0856
prob 4.72116963262411
error 0.7099176347113056
inital len 2
Customer 2 A caller hangs up at 31.0856
op2 len 1
Customer 1 is assigned to a operator2 at 35.2945
Customer 3 initiated a call at 36.7227
prob 2.6364871189000425
error 3.6364343198456606
inital len 2
Customer 1 is done operator2 at 40.0214
op1 len 1
Customer 3 is assigned to a operator1 at 45.2976
Customer 4 initiated a call at 50.8462
prob 9.74036933653065
error 7.536319904163672
inital len 2
Customer 5 initiated a call at 58.6753
prob 3.881958507575456
error 2.2840392442305713
inital len 3
op2 len 2
Customer 4 is assigned to a operator2 at 60.2958
Customer 4 is done operator2 at 63.4483
op2 len 1
Customer 5 is assigned to a operator2 at 63.4483

error 4.603667158059275
inital len 18
Operator2 break done at 915.923
break2 6
op2 len 17
Customer 154 is assigned to a operator2 at 918.523
Customer 154 is done operator2 at 921.481
Customer 156 initiated a call at 923.464
prob 3.320095960811711
error 6.089833169874083
inital len 18
Customer 157 initiated a call at 924.451
prob 8.92776941026858
error 9.91852421157852
inital len 19
op2 len 18
Customer 157 is assigned to a operator2 at 926.092
Customer 157 is done operator2 at 928.606
op2 len 17
Customer 156 is assigned to a operator2 at 928.606
Customer 146 is done operator1 at 930.389
Customer 145 is assigned to a operator1 at 930.389
Customer 156 is done operator2 at 932.632
op2 len 16
Customer 155 is assigned to a operator2 at 932.632
Customer 145 is done operator1 at 935.267
Customer 152 is assigned to a operator1 at 935.267
Customer 155 is done operator2 at 936.953
Customer 152 is done operator1 at 939.735
Operator1 gives break at 941.095
Operator1 break done at 944.095
break1 7
O

op2 len 29
Customer 285 is assigned to a operator2 at 1705.44
Customer 285 is done operator2 at 1707.73
op2 len 28
Customer 291 is assigned to a operator2 at 1709.85
Customer 291 is done operator2 at 1711.36
op1 len 27
Customer 292 initiated a call at 1716.86
prob 9.903785944286911
error 8.61811259762138
inital len 28
Customer 293 initiated a call at 1717.87
prob 1.3025368266886383
error 8.31390305656448
inital len 29
op1 len 28
op2 len 27
Customer 292 is assigned to a operator2 at 1734.77
Customer 292 is done operator2 at 1738.45
Customer 294 initiated a call at 1746.61
prob 8.98635188058596
error 9.95896376095368
inital len 28
Customer 295 initiated a call at 1748.11
prob 6.666867082678044
error 6.428403011432301
inital len 29
op2 len 28
Customer 295 is assigned to a operator2 at 1753.16
Customer 295 is done operator2 at 1757.62
op2 len 27
Customer 294 is assigned to a operator2 at 1757.62
Customer 294 is done operator2 at 1764.44
Customer 275 is done operator1 at 1771.59
Operator1 g

Customer 429 is done operator2 at 2656.96
op2 len 43
Customer 431 is assigned to a operator2 at 2656.96
Customer 432 initiated a call at 2657.54
prob 8.867614751555966
error 3.1363795530644745
inital len 44
Customer 431 is done operator2 at 2659.47
op2 len 43
Customer 430 is assigned to a operator2 at 2659.47
Customer 385 is done operator1 at 2662.4
Customer 388 is assigned to a operator1 at 2662.4
Customer 433 initiated a call at 2662.76
prob 2.657931168085894
error 8.171365152471411
inital len 44
op1 len 43
Customer 430 is done operator2 at 2664.04
op2 len 42
Customer 432 is assigned to a operator2 at 2664.04
Customer 434 initiated a call at 2667.88
prob 1.845557721971951
error 7.82826131067071
inital len 43
op1 len 42
Customer 435 initiated a call at 2669.81
prob 1.0075342667523435
error 0.5570714717663872
inital len 43
Customer 435 A caller hangs up at 2669.81
Customer 432 is done operator2 at 2670.44
Customer 436 initiated a call at 2681.62
prob 1.9359620368132735
error 0.11152181

error 2.9518872619578373
inital len 60
Customer 564 is done operator2 at 3508.56
Customer 568 initiated a call at 3509.74
prob 4.747645770881026
error 8.398755328939275
inital len 61
op2 len 60
Customer 568 is assigned to a operator2 at 3511.3
Customer 569 initiated a call at 3512.21
prob 2.0627706525601774
error 4.730439366057745
inital len 61
Customer 568 is done operator2 at 3515.14
Customer 570 initiated a call at 3515.42
prob 0.6161876174480985
error 0.2268942922560646
inital len 62
Customer 570 A caller hangs up at 3515.42
op1 len 61
op2 len 60
Customer 567 is assigned to a operator2 at 3521.7
Customer 567 is done operator2 at 3527.76
Customer 571 initiated a call at 3530.83
prob 0.6882217337561047
error 1.0014587740180214
inital len 61
Customer 572 initiated a call at 3531.12
prob 4.689048079430185
error 2.252758159976694
inital len 62
op2 len 61
Customer 572 is assigned to a operator2 at 3532.57
Customer 573 initiated a call at 3536.01
prob 0.8789843750612047
error 1.6635149579

...

break2 314
Operator1 gives break at 40613
Operator1 break done at 40616
break1 315
Operator2 gives break at 40642.7
Operator2 break done at 40645.7
break2 315
Operator1 gives break at 40672
Operator1 break done at 40675
break1 316
Operator2 gives break at 40714.8
Operator2 break done at 40717.8
break2 316
Operator1 gives break at 40734.8
Operator1 break done at 40737.8
break1 317
Operator2 gives break at 40756
Operator2 break done at 40759
break2 317
Operator1 gives break at 40779.7
Operator1 break done at 40782.7
break1 318
Operator2 gives break at 40793.7
Operator2 break done at 40796.7
break2 318
Operator1 gives break at 40817.8
Operator1 break done at 40820.8
break1 319
Operator2 gives break at 40917.8
Operator2 break done at 40920.8
break2 319
Operator1 gives break at 40944.1
Operator1 break done at 40947.1
break1 320
Operator2 gives break at 41016.6
Operator2 break done at 41019.6
break2 320
Operator1 gives break at 41022
Operator1 break done at 41025
break1 321
Operator2 gives b

Operator2 break done at 67830.2
break2 536
Operator1 gives break at 67863.7
Operator1 break done at 67866.7
break1 537
Operator2 gives break at 67898.5
Operator2 break done at 67901.5
break2 537
Operator1 gives break at 67905
Operator1 break done at 67908
break1 538
Operator2 gives break at 67910.4
Operator2 break done at 67913.4
break2 538
Operator1 gives break at 67921.8
Operator1 break done at 67924.8
break1 539
Operator2 gives break at 68037.1
Operator2 break done at 68040.1
break2 539
Operator1 gives break at 68051.8
Operator1 break done at 68054.8
break1 540
Operator2 gives break at 68178.6
Operator2 break done at 68181.6
break2 540
Operator1 gives break at 68221
Operator1 break done at 68224
break1 541
Operator2 gives break at 68337.1
Operator2 break done at 68340.1
break2 541
Operator1 gives break at 68378.8
Operator1 break done at 68381.8
break1 542
Operator2 gives break at 68421.5
Operator2 break done at 68424.5
break2 542
Operator1 gives break at 68445.9
Operator1 break done

break1 752
Operator2 gives break at 96060.1
Operator2 break done at 96063.1
break2 752
Operator1 gives break at 96176.7
Operator1 break done at 96179.7
break1 753
Operator2 gives break at 96283.4
Operator2 break done at 96286.4
break2 753
Operator1 gives break at 96388.2
Operator1 break done at 96391.2
break1 754
Operator2 gives break at 96482.9
Operator2 break done at 96485.9
break2 754
Operator1 gives break at 96638.4
Operator1 break done at 96641.4
break1 755
Operator2 gives break at 96787.5
Operator2 break done at 96790.5
break2 755
Operator1 gives break at 96885.9
Operator1 break done at 96888.9
break1 756
Operator2 gives break at 96988.7
Operator2 break done at 96991.7
break2 756
Operator1 gives break at 97053.9
Operator1 break done at 97056.9
break1 757
Operator2 gives break at 97066.7
Operator2 break done at 97069.7
break2 757
Operator1 gives break at 97075.1
Operator1 break done at 97078.1
break1 758
Operator2 gives break at 97150.6
Operator2 break done at 97153.6
break2 758
O

Operator1 gives break at 124225
Operator1 break done at 124228
break1 986
Operator2 gives break at 124237
Operator2 break done at 124240
break2 986
Operator1 gives break at 124310
Operator1 break done at 124313
break1 987
Operator2 gives break at 124325
Operator2 break done at 124328
break2 987
Operator1 gives break at 124584
Operator1 break done at 124587
break1 988
Operator2 gives break at 124616
Operator2 break done at 124619
break2 988
Operator1 gives break at 124623
Operator1 break done at 124626
break1 989
Operator2 gives break at 124667
Operator2 break done at 124670
break2 989
Operator1 gives break at 124774
Operator1 break done at 124777
break1 990
Operator2 gives break at 124829
Operator2 break done at 124832
break2 990
Operator1 gives break at 124833
Operator1 break done at 124836
break1 991
Operator2 gives break at 124855
Operator2 break done at 124858
break2 991
Operator1 gives break at 124861
Operator1 break done at 124864
break1 992
Operator2 gives break at 124916
Operat

KeyboardInterrupt: 